# Installation

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from transformers import BertTokenizer, BertForMaskedLM
from transformers import AdamW
from tqdm import tqdm  # for our progress bar
from torch.nn import functional as func
import torch
import pickle

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/curated-dataset/Toxic_NonToxic_Comments.csv
/kaggle/input/wordtoxicityscores/wordtoxicities.csv


# Data Import

In [ ]:
comments_df = pd.read_csv('/kaggle/input/curated-dataset/Toxic_NonToxic_Comments.csv', index_col = 0)
comments_df.head()

,comment,is_toxic
0,"""px solid #000080; background:#000000; padding...",0
1,Whistle Register\nWhen did she sing in the whi...,0
2,Dear AgentCDE\nGet bent.,0
3,some poor guy robs a liquor store for $400 and...,1
4,"Hey donkey \n\nHey donkey, your Dad is coming.",1


In [ ]:
wordtoxicities_df = pd.read_csv('/kaggle/input/wordtoxicityscores/wordtoxicities.csv', index_col = 0)
wordtoxicities_df.head()

,word,toxicity
0,aaaaaaaaaah,0.155084
1,ab,-0.089550
2,aba,-0.160136
3,abandon,-0.060261
4,abbasidumayyad,-0.050418


# BERT

The **BERT** model was pre-trained in English on the BookCorpus data, consisting of 11,038 books and other Wikipedia data, with lists, tables and headers excluded, so as to perform two interesting tasks, namely, Masked Language Modelling and Next Sentence Prediction. Our BERT model has been trained with the **MLM (Masked Language Modeling)** task, using the pre-trained BERT and fine-tuning its weights. Masked Language Modeling is a fill-in-the-blank task, where a model uses the context words surrounding a mask token to predict what the masked word should be. The masked word can then be substituted with the predicted word, generating a new sentence that shares similar context and same label with the original sentence. 

## Tokenizer 

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

BERT uses a special type of tokenizer called **WordPiece** tokenizer. The WordPiece tokenizer follows the subword tokenizer scheme. It's working can be understood by considering, as an example, the sentence: 'Let us start pretraining the model'. Tokenizing the sentence using WordPiece, 'token = \[let, us, start, pre, ###train, ###ing, the, model\]', is obtained. While tokenizing the sentence, the word 'pretraining', is split into three parts, because the WordPiece tokenizer first checks whether the word is present in its vocabulary; if the word is present then, it will be used as a token and if not, then, the word is split into subwords recursively until the subwords are found in the corpus. This process is effective in handling out-of-vocabulary words.

## Model 

In [ ]:
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


We are using **BERT-based uncased model** for Masked Language Modelling. MLM consists of giving BERT a sentence and optimizing the weights inside it to output the same sentence on the other side. However, before actually giving BERT the input sentence, a few tokens are masked. While in the original BERT model, the words are masked randomly, for our purpose, we are selecting those words that are associated with toxicity. 

## Revising Dataset because of constraints on Memory

Before tokenizing our dataset using BERT's Tokenizer, we are doing away with some amount of data. That is, those comments that have more tokens than **'max_tokens'** are rejected, so as to ensure that our model is successfully trained even with the limited memory available. 

In [ ]:
max_tokens = 64

valid_comments_df = pd.DataFrame(columns = ['comment', 'is_toxic'])

count_valid_comments = 0

for i in range(len(comments_df)):
    encoded_tokens = tokenizer.encode(comments_df['comment'][i], return_tensors = 'pt', max_length = max_tokens + 1, truncation = True)[0] 
    
    tokens = []
    for token in encoded_tokens:
        tokens.append(''.join(tokenizer.decode(token).split()))
    
    if(len(tokens) <= max_tokens):
        valid_comments_df.loc[len(valid_comments_df.index)] = comments_df['comment'][i], comments_df['is_toxic'][i]
        count_valid_comments += 1
        
print(count_valid_comments)

valid_comments_df.to_csv('Valid_Toxic_NonToxic_Comments.csv')

valid_comments_df

9735


,comment,is_toxic
0,"""px solid #000080; background:#000000; padding...",0
1,Whistle Register\nWhen did she sing in the whi...,0
2,Dear AgentCDE\nGet bent.,0
3,some poor guy robs a liquor store for $400 and...,1
4,"Hey donkey \n\nHey donkey, your Dad is coming.",1
...,...,...
9730,"this is the us , a trashy , misogynistic natio...",1
9731,WTF is Leif Erikson day \n\nand why does it de...,1
9732,"""\n\nSo I've been blocked for this which was t...",1
9733,For an Authentic account of India's untouchabl...,0


# Masking

## Tokenization 

BERT adds tokens like the the classifier token "\[CLS\]", which is the first token of a sequence, seperate token "\[SEP\]", which is the last token of a sequence, the unknown token "\[UNK\]" for tokens that are not in its vocabulary, and the token "\[PAD\]" used for padding, for batching sequences of different lengths, as in our case. It returns a list of token-type IDs according to the given sequence(s).  

In [ ]:
inputs = tokenizer(valid_comments_df.comment.tolist(), return_tensors = 'pt', max_length = max_tokens, truncation = True, padding = 'max_length')
inputs

{'input_ids': tensor([[  101,  1000,  1052,  ...,     0,     0,     0],
        [  101, 13300,  4236,  ...,     0,     0,     0],
        [  101,  6203,  4005,  ...,     0,     0,     0],
        ...,
        [  101,  1000,  2061,  ...,     0,     0,     0],
        [  101,  2005,  2019,  ...,     0,     0,     0],
        [  101,  1045,  3294,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

## Labels Tensor

We create our labels tensor by cloning the 'input_ids' tensor.

In [ ]:
inputs['labels'] = inputs.input_ids.detach().clone()

## Assigning Toxicity Scores to words known from BoW Toxicity Classifier

In [ ]:
wordtoxicities_dict = {}
for i in range(len(wordtoxicities_df)):
    wordtoxicities_dict[wordtoxicities_df['word'].tolist()[i]] = wordtoxicities_df['toxicity'].tolist()[i]
    
avg_word_toxicity = np.mean(wordtoxicities_df.toxicity.tolist())

comment_scores = []

for comment in valid_comments_df['comment']:
    encoded_tokens = tokenizer.encode(comment, return_tensors = 'pt', max_length = max_tokens + 1, truncation = True)[0] 
    
    tokens = []
    for token in encoded_tokens:
        tokens.append(''.join(tokenizer.decode(token).split()))
    
    token_scores = []
    
    for token in tokens:
        if token in wordtoxicities_dict:
            token_scores.append(wordtoxicities_dict[token])
        else:
            token_scores.append(avg_word_toxicity)        
    
    comment_scores.append(token_scores)

## Finding words to be Masked using an Adaptive Threshold

In [ ]:
min_threshold = 0.2

def find_masks_for_toxic_comment(index):
    scores = np.zeros(max_tokens)
    scores[:len(comment_scores[index])] = np.array(comment_scores[index])
    scores = torch.from_numpy(scores)
    
    threshold = max(min_threshold, max(comment_scores[index]) / 2)

    masks = (scores > threshold) * (inputs.input_ids[index] != 101) * (inputs.input_ids[index] != 102) * (inputs.input_ids[index] != 0)

    return masks

mask_arr = []
for i in range(len(valid_comments_df)):
    if(valid_comments_df['is_toxic'][i]):
        mask_arr.append(find_masks_for_toxic_comment(i))
    else:
        mask_arr.append(False * (inputs.input_ids[i] != 101) * (inputs.input_ids[i] != 102) * (inputs.input_ids[i] != 0))

## Applying Masks to all Comments

In [ ]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(torch.flatten(mask_arr[i].nonzero()).tolist())
    
selection

[[],
 [],
 [],
 [21],
 [1, 2, 3, 4],
 [5],
 [7, 20, 25, 43],
 [],
 [],
 [1, 12],
 [],
 [],
 [1, 6],
 [10, 20, 22],
 [1],
 [],
 [],
 [1],
 [],
 [7],
 [],
 [],
 [],
 [],
 [3],
 [5],
 [5, 6, 10],
 [13, 17],
 [4],
 [1],
 [],
 [],
 [],
 [],
 [9, 10],
 [],
 [5],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [9, 16],
 [],
 [],
 [1, 4],
 [24, 31],
 [8],
 [4],
 [2, 3, 6],
 [2, 3, 6],
 [3],
 [],
 [4],
 [],
 [1, 19],
 [],
 [],
 [],
 [],
 [],
 [3],
 [],
 [],
 [],
 [],
 [3, 9],
 [18],
 [6],
 [22],
 [],
 [],
 [20],
 [17],
 [],
 [1, 16],
 [],
 [],
 [6],
 [],
 [13],
 [9, 13],
 [],
 [5, 6],
 [2, 5],
 [8, 10, 11],
 [],
 [1],
 [10],
 [],
 [],
 [1],
 [18],
 [16],
 [],
 [4],
 [],
 [],
 [],
 [1, 2],
 [16],
 [],
 [],
 [],
 [],
 [3],
 [],
 [],
 [17],
 [],
 [],
 [],
 [],
 [10],
 [],
 [4],
 [2, 5, 8],
 [1, 2],
 [1, 2, 5, 6, 10, 11, 14, 15],
 [10],
 [6],
 [],
 [12],
 [6],
 [],
 [8],
 [],
 [],
 [1, 2],
 [12],
 [],
 [4],
 [14],
 [],
 [4],
 [3],
 [5],
 [],
 [15],
 [6],
 [7, 11],
 [],
 [7],
 [1],
 [8],
 [9],
 [1, 8],
 []

The values 103 are assigned the same positions as were found to hold 'True' values in the 'mask_arr' tensor:

In [ ]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

## Working 

We can see that toxic words in a given sequence are masked exactly as expected:

In [ ]:
for token_id in inputs.labels[9]:
    print(tokenizer.decode(token_id))

[ C L S ]
f u c k
t h a t
,
i
'
m
b e i n g
a c c
# # u s s
# # e d
o f
s h i t
t h a t
i
d i d n
'
t
d o
[ S E P ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]


In [ ]:
print(max(min_threshold, max(comment_scores[9])/2))
print(comment_scores[6])

5.534921985953844
[0.005315426966458784, 0.005315426966458784, 0.005315426966458784, 0.005315426966458784, 0.005315426966458784, 5.0663795128326425, 0.005315426966458784, 11.069843971907687, 0.005315426966458784, 1.309723401308056, 0.005315426966458784, 0.005315426966458784, 0.2037369385745014, 0.005315426966458784, -0.1681766601614798, 0.005315426966458784, 0.005315426966458784, 2.841683505223333, 0.005315426966458784, 0.005315426966458784, 11.069843971907687, 0.005315426966458784, 0.2969538184224775, 0.005315426966458784, -0.2306175113953349, 11.069843971907687, 0.005315426966458784, -0.2306175113953349, 0.0825115595062786, 0.4131274776827482, 0.005315426966458784, 0.005315426966458784, 0.005315426966458784, 0.005315426966458784, 0.005315426966458784, 0.005315426966458784, 0.005315426966458784, 0.005315426966458784, 0.005315426966458784, 0.005315426966458784, -0.1442488831252684, 0.005315426966458784, -0.1240947637480212, 9.211758592421704, 0.005315426966458784, 0.005315426966458784,

In [ ]:
for token_id in inputs.input_ids[9]:
    print(tokenizer.decode(token_id))

[ C L S ]
[ M A S K ]
t h a t
,
i
'
m
b e i n g
a c c
# # u s s
# # e d
o f
[ M A S K ]
t h a t
i
d i d n
'
t
d o
[ S E P ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]
[ P A D ]


# Processing

## Preparing Dataset

In [ ]:
class ToxicDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
dataset = ToxicDataset(inputs)

In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size = 32)

## Setting-up

Before moving onto setting-up the training loop, GPU/CPU usage is set up.

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#and moving our model over to the selected device
model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

## Optimizing

Activating the training mode of our model, and initializing our optimizer: Adam with weighted decay that reduces chances of overfitting.

In [ ]:
#activating training mode
model.train()
#initializing optimizer
optim = AdamW(model.parameters(), lr = 5e-5)

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


## Training the Model

In [ ]:
epochs = 2

for epoch in range(epochs):
    #setting up loop with TQDM and dataloader
    loop = tqdm(loader, leave = True)
    for batch in loop:
        #initializing calculated gradients (from previous step)
        optim.zero_grad()
        #pulling all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        #processing
        outputs = model(input_ids, attention_mask = attention_mask, labels = labels)
        print(outputs[0])
        #extracting loss
        loss = outputs.loss
        #calculating loss for every parameter that needs grad update
        loss.backward()
        #updating parameters
        optim.step()
        #printing relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss = loss.item())

  0%|          | 0/305 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


tensor(13.6442, grad_fn=<NllLossBackward>)


Epoch 0:   0%|          | 1/305 [00:14<1:14:17, 14.66s/it, loss=13.6]

tensor(11.0597, grad_fn=<NllLossBackward>)


Epoch 0:   1%|          | 2/305 [00:28<1:11:57, 14.25s/it, loss=11.1]

...

Epoch 1:  99%|█████████▉| 303/305 [1:00:40<00:24, 12.14s/it, loss=0.0311]

tensor(0.0198, grad_fn=<NllLossBackward>)


Epoch 1: 100%|█████████▉| 304/305 [1:00:52<00:12, 12.05s/it, loss=0.0198]

tensor(0.0388, grad_fn=<NllLossBackward>)


Epoch 1: 100%|██████████| 305/305 [1:00:55<00:00, 11.98s/it, loss=0.0388]


# Output

The output of the BERT model is the tensor **logits**. After passing the input-encoding into the BERT model and training the model, the tensor logits is obtained simply by specifying output.logits. And then, a SoftMax activation function is finally applied to the logits. By applying a **SoftMax** onto the output of BERT, probabilistic distributions for each of the words in BERT’s vocabulary are obtained for replacing the mask token. Words with a higher probability value will be better candidate replacement words for the mask token. However, despite using class-specific sentence embeddings, BERT often predicts toxic words, apparently paying more attention to the context than to the embeddings of the desired class. And so, to force the model to generate non-toxic words, the toxicity of each token in BERT vocabulary is calculated and is used to penalize the predicted probabilities. 

In order to get the tensor of SoftMax values of all the words in BERT’s vocabulary, one can get the masked token index using torch.where(). The torch.topk() function, which allows one to retrieve the top k values in a given tensor, and returns a tensor containing the top k values, is iterated through, and the mask token in the sentence is replaced with the candidate tokens. Instead of using torch.topk() for retrieving the top l values, torch.argmax(), which returns the index of the maximum value in the tensor, can also be used.

In [ ]:
max_word_toxicity = np.max(wordtoxicities_df.toxicity.tolist())
min_word_toxicity = np.min(wordtoxicities_df.toxicity.tolist())

#toxic comment

unmasked = "Your [MASK] is your [MASK]!";
masked = tokenizer.encode(unmasked, return_tensors = 'pt')[0] 

comment = []
for token in masked:
    comment.append(''.join(tokenizer.decode(token).split()))

#retrieving detoxified comment

mask_token_indices = torch.where(masked == tokenizer.mask_token_id)[0]

token_logits = outputs.logits

softy = func.softmax(token_logits, dim = -1)
mask_token_logits = softy[0, mask_token_indices, :]

for i in range(len(mask_token_indices.tolist())):
    top_ten_tokens = torch.topk(mask_token_logits, 10, dim = 1, sorted = True).indices[i].tolist()
    
    #before rescoring on the basis of toxicities
    for token in top_ten_tokens:
        comment[mask_token_indices[i]] = tokenizer.decode(token)
        print(comment)

    print()
    
    mask_tokens =  torch.sort(mask_token_logits, dim = 1, stable = True).indices[i].tolist()
    token_logits = torch.sort(mask_token_logits, dim = 1, stable = True).values[i].tolist()
        
    better_mask_tokens = []
    better_mask_token_logits = []
    for j in range(len(token_logits)):
        better_mask_tokens.append(mask_tokens[j])

        token = ''.join(tokenizer.decode(mask_tokens[j]).split())
        if token in wordtoxicities_dict:
            better_mask_token_logits.append(token_logits[j] / (max_word_toxicity - min_word_toxicity + wordtoxicities_dict[token]))
        else:
            better_mask_token_logits.append(token_logits[j] / (max_word_toxicity - min_word_toxicity + avg_word_toxicity))

    better_mask_tokens.sort(key = dict(zip(better_mask_tokens, better_mask_token_logits)).get)
    better_top_ten_tokens = better_mask_tokens[-10: ]
    
    #after rescoring on the basis of toxicities
    for token in better_top_ten_tokens:
        comment[mask_token_indices[i]] = tokenizer.decode(token)
        print(comment)
    
    print()

['[CLS]', 'your', 'v o t e', 'is', 'your', '[MASK]', '!', '[SEP]']
['[CLS]', 'your', 'v o t e s', 'is', 'your', '[MASK]', '!', '[SEP]']
['[CLS]', 'your', 'v o t e d', 'is', 'your', '[MASK]', '!', '[SEP]']
['[CLS]', 'your', 'v o t i n g', 'is', 'your', '[MASK]', '!', '[SEP]']
['[CLS]', 'your', 'c o u n t', 'is', 'your', '[MASK]', '!', '[SEP]']
['[CLS]', 'your', 't h i n k', 'is', 'your', '[MASK]', '!', '[SEP]']
['[CLS]', 'your', 'v e', 'is', 'your', '[MASK]', '!', '[SEP]']
['[CLS]', 'your', 'g r a m', 'is', 'your', '[MASK]', '!', '[SEP]']
['[CLS]', 'your', 'f e e l', 'is', 'your', '[MASK]', '!', '[SEP]']
['[CLS]', 'your', '# # v e', 'is', 'your', '[MASK]', '!', '[SEP]']

['[CLS]', 'your', '# # v e', 'is', 'your', '[MASK]', '!', '[SEP]']
['[CLS]', 'your', 'f e e l', 'is', 'your', '[MASK]', '!', '[SEP]']
['[CLS]', 'your', 'g r a m', 'is', 'your', '[MASK]', '!', '[SEP]']
['[CLS]', 'your', 'v e', 'is', 'your', '[MASK]', '!', '[SEP]']
['[CLS]', 'your', 't h i n k', 'is', 'your', '[MASK]', '!

# Saving Output

In [ ]:
torch.save(outputs.logits, "BERTLogits.pt")

<a href = "./BERTLogits.pt"> Download File </a>

# References

1. Understanding the BERT Model
   *https://medium.com/analytics-vidhya/understanding-the-bert-model-a04e1c7933a9*
   
2. Hugging Face Transformer library: BERT Documentation
   *https://huggingface.co/docs/transformers/model_doc/bert*

3. Complete Tutorial on BERT
   *https://analyticsindiamag.com/a-complete-tutorial-on-masked-language-modelling-using-bert/*

4. Training for MLM Task
   *https://github.com/jamescalam/transformers/blob/main/course/training/03_mlm_training.ipynb*

5. How to use BERT
   *https://towardsdatascience.com/how-to-use-bert-from-the-hugging-face-transformer-library-d373a22b0209*